In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
class LogisticRegression:
  def __init__(self, learning_rate = 0.01, epochs = 100):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.weights = None
    self.bias = None
    self.length = None
    self.dimensions = None

  def linear(self, X):
    return torch.matmul(X, self.weights) + self.bias

  def fit(self, X, y):
    self.length, self.dimensions = X.shape
    torch.manual_seed(0)
    self.weights = 2 * torch.rand(self.dimensions) - 1
    self.bias = 0
    for epoch in range(self.epochs):
      self.update_gradient(X, y)
      if epoch % 10 == 0:
          loss = torch.nn.functional.binary_cross_entropy(self.predict(X), y)
          print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

  def predict(self, X):
    z = self.linear(X)
    y_pred = torch.sigmoid(z)
    return y_pred

  def predict_exact(self, X, threshold):
    z = self.linear(X)
    y_pred = torch.sigmoid(z)
    return (y_pred >= threshold).float()

  def update_gradient(self, X, y):
    error = self.predict(X) - y
    self.weights -= self.learning_rate * torch.matmul(X.T, error) / self.length
    self.bias -= self.learning_rate * torch.sum(error) / self.length

In [ ]:
class OneVsAllLogisticRegression:
  def __init__(self, num_classes, learning_rate = 0.01, epochs = 100):
    self.models = []
    for i in range(num_classes):
      self.models.append(LogisticRegression(learning_rate = learning_rate, epochs = epochs))

  def fit(self, X, y):
    for i in range(len(self.models)):
      y_encoded = (y[:,i]).float()
      self.models[i].fit(X, y_encoded)

  def predict(self, X):
    preds = torch.stack([model.predict(X).squeeze() for model in self.models])
    return torch.argmax(preds, dim=0)

In [ ]:
# upgraded version; Logistic Regression implemented by chatGPT
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None
        self.length = None
        self.dimensions = None
        self.loss_history = []

    def linear(self, X):
        return torch.matmul(X, self.weights) + self.bias

    def fit(self, X, y):
        # Ensure inputs are tensors
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32)
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y, dtype=torch.float32)

        self.length, self.dimensions = X.shape

        # Initialize weights and bias
        torch.manual_seed(0)
        self.weights = torch.randn(self.dimensions) * 0.1  # Better initialization
        self.bias = torch.tensor(0.0)

        self.loss_history = []

        for epoch in range(self.epochs):
            self.update_gradient(X, y)

            if epoch % 10 == 0:
                with torch.no_grad():
                    predictions = self.predict(X)
                    # Add small epsilon to prevent log(0)
                    predictions = torch.clamp(predictions, 1e-7, 1 - 1e-7)
                    loss = F.binary_cross_entropy(predictions, y)
                    self.loss_history.append(loss.item())
                    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    def predict(self, X):
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32)
        z = self.linear(X)
        y_pred = torch.sigmoid(z)
        return y_pred

    def predict_exact(self, X, threshold=0.5):
        predictions = self.predict(X)
        return (predictions >= threshold).float()

    def update_gradient(self, X, y):
        predictions = self.predict(X)
        error = predictions - y

        # Gradient descent updates
        self.weights -= self.learning_rate * torch.matmul(X.T, error) / self.length
        self.bias -= self.learning_rate * torch.sum(error) / self.length


class MultinomialLogisticRegression:
    def __init__(self, num_classes, learning_rate=0.01, epochs=100):
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None
        self.loss_history = []

    def _one_hot_encode(self, y):
        if len(y.shape) == 1:
            y_encoded = torch.zeros(len(y), self.num_classes)
            y_encoded[torch.arange(len(y)), y.long()] = 1
            return y_encoded
        else:
            return y.float()

    def fit(self, X, y):
        # Ensure inputs are tensors
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32)
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y)

        # Convert to one-hot if necessary
        y_encoded = self._one_hot_encode(y)

        length, dimensions = X.shape

        # Initialize weights and bias
        torch.manual_seed(0)
        self.weights = torch.randn(dimensions, self.num_classes) * 0.1
        self.bias = torch.zeros(self.num_classes)

        self.loss_history = []

        for epoch in range(self.epochs):
            self._update_gradient(X, y_encoded)

            if epoch % 10 == 0:
                with torch.no_grad():
                    predictions = self.predict_proba(X)
                    loss = F.cross_entropy(predictions, torch.argmax(y_encoded, dim=1))
                    self.loss_history.append(loss.item())
                    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    def _linear(self, X):
        return torch.matmul(X, self.weights) + self.bias

    def predict_proba(self, X):
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32)
        z = self._linear(X)
        return F.softmax(z, dim=1)

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return torch.argmax(probabilities, dim=1)

    def _update_gradient(self, X, y_encoded):
        probabilities = self.predict_proba(X)
        error = probabilities - y_encoded

        length = X.shape[0]

        # Update weights and bias
        self.weights -= self.learning_rate * torch.matmul(X.T, error) / length
        self.bias -= self.learning_rate * torch.sum(error, dim=0) / length


In [ ]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
df = pd.read_csv(url)

In [ ]:
temp = df[df["species"].isin(["setosa", "virginica"])]

In [ ]:
X, y = temp.iloc[:,:4], temp.iloc[:,4]

In [ ]:
X = torch.tensor(X.values, dtype = torch.float32)

In [ ]:
y_encoded = torch.tensor(pd.get_dummies(y, drop_first=True).values, dtype = torch.float32).squeeze()

In [ ]:
model = LogisticRegression(learning_rate= 0.01, epochs = 500)

In [ ]:
model.fit(X, y_encoded)

Epoch 0, Loss: 2.6351
Epoch 10, Loss: 1.9539
Epoch 20, Loss: 1.6256
Epoch 30, Loss: 1.4381
Epoch 40, Loss: 1.2800
Epoch 50, Loss: 1.1374
Epoch 60, Loss: 1.0097
Epoch 70, Loss: 0.8970
Epoch 80, Loss: 0.7987
Epoch 90, Loss: 0.7135
Epoch 100, Loss: 0.6400
Epoch 110, Loss: 0.5767
Epoch 120, Loss: 0.5223
Epoch 130, Loss: 0.4754
Epoch 140, Loss: 0.4347
Epoch 150, Loss: 0.3995
Epoch 160, Loss: 0.3687
Epoch 170, Loss: 0.3417
Epoch 180, Loss: 0.3180
Epoch 190, Loss: 0.2969
Epoch 200, Loss: 0.2782
Epoch 210, Loss: 0.2615
Epoch 220, Loss: 0.2465
Epoch 230, Loss: 0.2330
Epoch 240, Loss: 0.2208
Epoch 250, Loss: 0.2097
Epoch 260, Loss: 0.1996
Epoch 270, Loss: 0.1904
Epoch 280, Loss: 0.1819
Epoch 290, Loss: 0.1741
Epoch 300, Loss: 0.1669
Epoch 310, Loss: 0.1602
Epoch 320, Loss: 0.1541
Epoch 330, Loss: 0.1484
Epoch 340, Loss: 0.1430
Epoch 350, Loss: 0.1380
Epoch 360, Loss: 0.1334
Epoch 370, Loss: 0.1290
Epoch 380, Loss: 0.1249
Epoch 390, Loss: 0.1210
Epoch 400, Loss: 0.1174
Epoch 410, Loss: 0.1140
Epo

In [ ]:
X, y = df.iloc[:,:4], df.iloc[:,4]

In [ ]:
X = torch.tensor(X.values, dtype = torch.float32)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
X = torch.tensor(X, dtype = torch.float32)

In [ ]:
y_encoded = torch.tensor(pd.get_dummies(df.iloc[:,4]).values, dtype = torch.float32)

In [ ]:
model = MultinomialLogisticRegression(3, learning_rate= 0.01, epochs = 100)

In [ ]:
model.fit(X, y_encoded)

Epoch 0, Loss: 1.1234
Epoch 10, Loss: 1.0902
Epoch 20, Loss: 1.0597
Epoch 30, Loss: 1.0322
Epoch 40, Loss: 1.0077
Epoch 50, Loss: 0.9862
Epoch 60, Loss: 0.9672
Epoch 70, Loss: 0.9506
Epoch 80, Loss: 0.9359
Epoch 90, Loss: 0.9230


In [ ]:
model.predict(X)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1,
        2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1,
        1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [ ]:
y.apply(lambda x: map[x])

,species
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


In [ ]:
predictions[0]

tensor(0)

In [ ]:
actual = torch.tensor(y.apply(lambda x: map[x]))

In [ ]:
actual[0]

tensor(0)

In [ ]:
total_sum = 0
predictions = model.predict(X)
print((actual == predictions).sum() / len(actual))

tensor(0.8333)


In [ ]:
map = {"setosa" : 0, "virginica" : 2, "versicolor" : 1 }

In [ ]:
X[0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [ ]:
y.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data
y = iris.target

# Scale features
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

Accuracy: 1.0


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
